In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://www.w3schools.com/w3js/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [3]:
print(status)

200


In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> W3.JS
h1 -> W3.JS Tutorial
h2 -> What Can W3.JS Do?
h2 -> W3.JS Examples
h2 -> W3.JS References
h2 -> How To
h3 -> Example
h3 -> Example
h2 -> Report Error
h2 -> Thank You For Helping Us!


In [6]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left">W3.JS</h2>]

In [7]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

W3.JS


In [8]:
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

In [9]:
# getting the Headings, Sub-Headings, Links and Contents

# CREATING RELEVANT VARIABLES
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []
ordered_lists = []
list_headings = []
examples_per_page = []
excercises = []
base_url = "https://www.w3schools.com/w3js/"
course_url = "https://www.w3schools.com/w3js/"

# FINDING ALL THE LINKS PRESENT IN THE LEFT MENU AND ITERATING OVER THE LINKS TO SCRAPE THE DATA
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
           
            if extra is not None :
                print(extra)
                if extra[0] == '/':
                    base_url = "https://www.w3schools.com"
                    
            elif extra is None :
                continue 
           
                
                
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            page_examples = soup_page.find_all('div', class_ = 'w3-code notranslate htmlHigh')
            
            ##################################################################################################
            
            # Examples per page
            pg_example = []
            for example in page_examples:
                nw = []
                for i in list(example):
                    if str(i) == '<br/>':
                        pass
                    else:
                        nw.append(i)
                pg_example.append(nw)
            examples_per_page.append(pg_example)
                
            ##################################################################################################
            
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            if len(row) !=0:
                if row[0] == '&':
                    row = row[3:]
            table_new.append(row)
            table_string = ''
            for each in table_new:
                table_string += each + '$$$'
            table_new = table_string[:-3]
            page_table.append(table_new)
            
            ##################################################################################################
            
            # Points for Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                unorder.append(start_string)
            if len(unorder) == 0:
                unorder = None
            else:
                unorder_string = ''
                for i in unorder:
                    unorder_string += i +'|||'
                unorder = unorder_string[:-3]
            unordered_lists.append(unorder)
            
                        
            ##################################################################################################
            #Points for ordered list.
            ol = soup_page.find_all('ol')
            order = []
            for i in ol:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                order.append(start_string)
            
            if len(order) == 0:
                order = None
            else:
                order_string = ''
                for i in order:
                    order_string += i +'|||'
                order = order_string[:-3]
            ordered_lists.append(order)
            
            ##################################################################################################
            
            # Headings and Subheadings and Description
            
            list_headings_per_page = []
            for page_data in page_content:
                for j in str(page_data).split('\n'):
                    list_headings_per_page.append(j)
                page_paragraphs = page_data.find_all('p')
        
            content_per_page = ''
            for i in page_paragraphs:
                content_per_page += i.text + '|||'
            content_per_page = content_per_page[:-3].replace('\n', '')
            sub_page_content_list.append(content_per_page)
            
            prev = ''
            ul_ol_heading = ''
            for i in list_headings_per_page:
                if ('ul' in i) and (('h2' in prev) or ('h3' in prev)):
                    ul_ol_heading += prev[4:-5] + '|||'
                prev = i
            list_headings.append(ul_ol_heading[:-3])
#             time.sleep(5)      

default.asp
w3js_intro.asp
w3js_selectors.asp
w3js_hide.asp
w3js_add_style.asp
w3js_add_class.asp
w3js_filters.asp
w3js_sort.asp
w3js_slideshow.asp
w3js_html_include.asp
w3js_display.asp
w3js_http.asp
w3js_controllers.asp
w3js_servers.asp
w3js_w3data.asp
w3js_examples.asp
w3js_references.asp
w3js_downloads.asp


In [10]:
length = len(course_links)
length

18

In [11]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(list_headings), len(unordered_lists),len(ordered_lists),  len(page_table), len(examples_per_page)

(18, 18, 18, 18, 18, 18, 18, 18, 18)

In [12]:
from csv import writer

# writing the file
with open('w3js_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('w3js_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,W3.JS,W3.JS HOME,https://www.w3schools.com/w3js/default.asp,W3.JS is a JavaScript library designed to simp...,NaN,NaN,NaN,NaN,"[['\n<script src=""https://www.w3schools.com/li..."
1,W3.JS,W3.JS Intro,https://www.w3schools.com/w3js/w3js_intro.asp,Click to Display Data|||Click on the table hea...,NaN,"Name@@@{{CustomerName}}, {{Country}}|||Alfreds...",NaN,Berglunds snabbköp|||Sweden&&&North/South|||UK...,[]
2,W3.JS,W3.JS Selectors,https://www.w3schools.com/w3js/w3js_selectors.asp,W3.JS selects HTML elements and perform action...,NaN,The action() is performed on the selected elem...,NaN,"(""*"")|||Selects all elements in the document||...",[]
3,W3.JS,W3.JS Hide/Show,https://www.w3schools.com/w3js/w3js_hide.asp,"To hide an element with a specified id, use a ...",NaN,NaN,NaN,NaN,"[['\n <button onclick=""w3.hide(\'#London\')"">..."
4,W3.JS,W3.JS Add Style,https://www.w3schools.com/w3js/w3js_add_style.asp,Add a CSS property value:|||Add a red backgrou...,NaN,NaN,NaN,NaN,"[['\n <button onclick=""w3.addStyle(\'#London\..."


In [13]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

C:\Users\NITHIN~1\AppData\Local\Temp/ipykernel_5576/1246472655.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Examples'][i] = None


In [14]:
from csv import writer

# writing the file
with open('w3js_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('w3js_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,W3.JS,W3.JS HOME,https://www.w3schools.com/w3js/default.asp,W3.JS is a JavaScript library designed to simp...,NaN,NaN,NaN,NaN,"[['\n<script src=""https://www.w3schools.com/li..."
1,W3.JS,W3.JS Intro,https://www.w3schools.com/w3js/w3js_intro.asp,Click to Display Data|||Click on the table hea...,NaN,"Name@@@{{CustomerName}}, {{Country}}|||Alfreds...",NaN,Berglunds snabbköp|||Sweden&&&North/South|||UK...,[]
2,W3.JS,W3.JS Selectors,https://www.w3schools.com/w3js/w3js_selectors.asp,W3.JS selects HTML elements and perform action...,NaN,The action() is performed on the selected elem...,NaN,"(""*"")|||Selects all elements in the document||...",[]
3,W3.JS,W3.JS Hide/Show,https://www.w3schools.com/w3js/w3js_hide.asp,"To hide an element with a specified id, use a ...",NaN,NaN,NaN,NaN,"[['\n <button onclick=""w3.hide(\'#London\')"">..."
4,W3.JS,W3.JS Add Style,https://www.w3schools.com/w3js/w3js_add_style.asp,Add a CSS property value:|||Add a red backgrou...,NaN,NaN,NaN,NaN,"[['\n <button onclick=""w3.addStyle(\'#London\..."


In [15]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

C:\Users\NITHIN~1\AppData\Local\Temp/ipykernel_5576/1246472655.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Examples'][i] = None


In [16]:
dataset = dataset.replace(np.nan, '', regex=True)

In [17]:
# Making a separate column for those heading which are not associated with the Lists
new_col = [None for i in range(len(course_links))]
dataset['Other Headings'] = pd.Series(new_col)
for i in range(len(dataset['Unordered Lists'])):
    if dataset['Unordered Lists'][i] == '':
        dataset['Other Headings'][i] = dataset['List Headings'][i]
        dataset['List Headings'][i] = ''

In [18]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)
dataset.head(10)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples,Other Headings
0,W3.JS,W3.JS HOME,https://www.w3schools.com/w3js/default.asp,W3.JS is a JavaScript library designed to simp...,,,,,"[['\n<script src=""https://www.w3schools.com/li...",
1,W3.JS,W3.JS Intro,https://www.w3schools.com/w3js/w3js_intro.asp,Click to Display Data|||Click on the table hea...,,"Name@@@{{CustomerName}}, {{Country}}|||Alfreds...",,Berglunds snabbköp|||Sweden&&&North/South|||UK...,,
2,W3.JS,W3.JS Selectors,https://www.w3schools.com/w3js/w3js_selectors.asp,W3.JS selects HTML elements and perform action...,,The action() is performed on the selected elem...,,"(""*"")|||Selects all elements in the document||...",,
3,W3.JS,W3.JS Hide/Show,https://www.w3schools.com/w3js/w3js_hide.asp,"To hide an element with a specified id, use a ...",,,,,"[['\n <button onclick=""w3.hide(\'#London\')"">...",
4,W3.JS,W3.JS Add Style,https://www.w3schools.com/w3js/w3js_add_style.asp,Add a CSS property value:|||Add a red backgrou...,,,,,"[['\n <button onclick=""w3.addStyle(\'#London\...",
5,W3.JS,W3.JS Add Class,https://www.w3schools.com/w3js/w3js_add_class.asp,Add a class:|||Add multiple classes:|||Add the...,,,,,"[['\n <button onclick=""w3.addClass(\'#London\...",Add Multiple Classes|||Remove Multiple Classes
6,W3.JS,W3.JS Filters,https://www.w3schools.com/w3js/w3js_filters.asp,Filter elements:|||Search for a name in the in...,,{{CustomerName}},,Berglunds snabbköp|||Sweden&&&North/South|||UK...,"[['\n <input oninput=""w3.filterHTML(\'#id01\'...",
7,W3.JS,W3.JS Sort,https://www.w3schools.com/w3js/w3js_sort.asp,Sort elements:|||Click the button to sort the ...,,Oslo@@@Stockholm@@@Helsinki@@@Berlin@@@Rome@@@...,,Berglunds snabbkop|||Sweden&&&North/South|||UK...,"[['\n <button onclick=""w3.sortHTML(\'#id01\',...",
8,W3.JS,W3.JS Slideshow,https://www.w3schools.com/w3js/w3js_slideshow.asp,Start slideshow:|||To prevent the slideshow fr...,,,,,"[['\n<img class=""nature"" src=""img_snowtops.jpg...",
9,W3.JS,W3.JS Includes,https://www.w3schools.com/w3js/w3js_html_inclu...,Include an HTML file:|||Save the HTML you want...,,,,,"[['\n <a href=""https://www.w3schools.com/html/...",
